In [1]:
import json
import plotly.express as px
import pandas as pd

In [2]:
json_file = json.load(open('data_level0.json', encoding="utf8"))
keys = json_file.keys()
print(keys)

dict_keys(['dev', 'test', 'train'])


In [3]:
def extract_data(key: str, json_file: json) -> dict:
    print(f"Total number of Utterances: {len(json_file[key])}")

for key in keys:
    print(key)
    extract_data(key, json_file)
    print("\n\n")

dev
Total number of Utterances: 1108



test
Total number of Utterances: 2610



train
Total number of Utterances: 9989





# Pre-processing Rules

- Replace '’' with '''. (Issue: mixture of apostrophe)

# Layer 1

- Count the distribution of emotion
- Count the number of sentiments

In [4]:
emotion_count = {}
sentiment_count = {}
dialogue_dict = {}
utterance_length = []

def concat_dialogue_dict(dialogue_dict: dict, dialog_id: str, utterance: str) -> None:
    
    # Check if
    dialogue_dict[dialog_id] = dialogue_dict.get(dialog_id, 0) + 1
    
    return None

def preprocessing_utterance(utterance: str) -> str:
    utterance = utterance.replace("’","'")
    
    return utterance

# Use Train dataset for testing
for key in ['train']:
    
    # Extract all the dialogue in given array
    # dia0_utt0 -> Dialogue 0, Utterance 0
    utterance_dialogue_list = json_file[key]
    
    # Go through one utterance at a time
    for utterance_id in utterance_dialogue_list.keys():
        utter_dict = utterance_dialogue_list[utterance_id]
        
        # Preprocessing
        utterance = preprocessing_utterance(utter_dict['Utterance'])
        
        # Add utterance to dialog dictionary, due dataset structure.
        concat_dialogue_dict(dialogue_dict, utter_dict['Dialogue_ID'], utterance)
        
        # Find the length of an utterance
        utterance_tokens = utterance.split(" ")
        utterance_length += [len(utterance_tokens)]
        
        # For emotion and sentiment labels
        emotion = utter_dict['Emotion']
        sentiment = utter_dict['Sentiment']
        
        emotion_count[emotion] = emotion_count.get(emotion, 0) + 1
        sentiment_count[sentiment] = sentiment_count.get(sentiment, 0) + 1

In [14]:
emotion_count_df = pd.DataFrame(emotion_count.items(), columns=['emotion', 'count'])

fig = px.bar(emotion_count_df, x='emotion', y='count', title='Emotion Counts')
fig.update_layout(hovermode="x")
fig.show()

In [11]:
sentiment_count_df = pd.DataFrame(sentiment_count.items(), columns=['sentiment', 'count'])

fig = px.bar(sentiment_count_df, x='sentiment', y='count', title='Topic Counts')
fig.update_layout(hovermode="x")
fig.show()

# Layer 2

- Count the number of utterances and distribution (Histogram)
- Count the number of tokens and distribution (Histogram)

In [13]:
dialogue_df = pd.DataFrame(dialogue_dict.values(), columns=['utterance_per_dialogue'])

fig = px.histogram(dialogue_df, x="utterance_per_dialogue")
fig.update_layout(bargap=0.5, hovermode="x unified")
fig.show()

In [12]:
utterance_length_df = pd.DataFrame(utterance_length, columns=['utterance_length'])

fig = px.histogram(utterance_length_df, x="utterance_length")
fig.update_layout(bargap=0.5, hovermode="x")
fig.show()

# Self-Analysis

- Starting line number in JSON: Dev(2), Train(48340), Valid(14408)
- There are 1038 dialogues in the training dataset, which consists of 9989 utterances
- The utterances needs to be re-sorted, as the dataset utterances are in random order (does not follow sequence)
- There are 80,258 tokens from 9989 utterances in the training dataset

# Layer 3

- Break down sentiments to find distribution of emotions
- Find the number of speakers per dialog
- Cross check utterance length and emotion (Even number length vs odd number length)
- Distribution of total utterance length in one dialog
- Cross check utterance length and emotions
- Check the nature of dialogue (DailyDialog (Simulated) vs MELD vs EmoryNLP) -> More in-depth analysis